In [ ]:
pip install tensorflow

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories and parameters
dataset_combined_dir = 'plant dataset'
target_size = (224, 224)  # Target image size for MobileNetV2
batch_size = 32
epochs = 10

In [2]:
# Use ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [3]:
# Load data from combined directory
train_generator = datagen.flow_from_directory(
    dataset_combined_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1320 images belonging to 16 classes.


In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load MobileNetV2 base model
base_model = MobileNetV2(input_shape=(target_size[0], target_size[1], 3), include_top=False, weights='imagenet')

In [5]:
# Add custom top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [6]:
# Combine base model and custom top layers into a new model
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Freeze layers in base model (optional)
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Train the model
model.fit(
    train_generator,
    epochs=10
)

Epoch 1/10


C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


42/42 ━━━━━━━━━━━━━━━━━━━━ 116s 2s/step - accuracy: 0.5245 - loss: 1.6732
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.9295 - loss: 0.2351
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.9647 - loss: 0.1291
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.9704 - loss: 0.0964
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.9857 - loss: 0.0623
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 76s 2s/step - accuracy: 0.9783 - loss: 0.0663
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.9844 - loss: 0.0623
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.9933 - loss: 0.0325
Epoch 9/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.9939 - loss: 0.0296
Epoch 10/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.9822 - loss: 0.0547


In [10]:
# Save the trained model
model.save('final_model.keras')

In [11]:
#make predictions
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('final_model.keras')

# Define the labels for medicinal and toxic plants
medicinal_labels = [
    "Aloevera", "Amruthaballi", "Betal", "Doddpathre", "Eucalyptus","Neem","Tulsi","Turmeric"
]

toxic_labels = [
    "castor_oil_plant", "dieffenbachia", "foxglove", "lilies", "lily_of_the_valley","oleander","rhubarb","wisteria"
]

# Combine labels
class_labels = medicinal_labels + toxic_labels

# Function to preprocess an image
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)  # Load and resize image
    img_array = image.img_to_array(img)  # Convert to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to create a batch-like effect
    img_array = img_array / 255.0  # Normalize the image
    return img_array

# Function to predict and print category
def predict_and_print_category(image_path):
    img_array = preprocess_image(image_path)
    
    # Make predictions
    predictions = model.predict(img_array)
    
    # Print the predicted class probabilities
    print("Predicted class probabilities:", predictions)
    
    # Print the predicted class index
    predicted_class = np.argmax(predictions, axis=1)[0]
    print(f'Predicted class index: {predicted_class}')
    
    # Determine if the class is medicinal or toxic
    if predicted_class < 5:
        print(f"The image is classified as a medicinal plant: {class_labels[predicted_class]}")
    else:
        print(f"The image is classified as a toxic plant: {class_labels[predicted_class]}")

# Example: Predict on a new image from the combined dataset
new_image_path = 'plant dataset/lilies/147801.jpg'
predict_and_print_category(new_image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted class probabilities: [[1.4791154e-05 7.7362819e-07 5.4048123e-06 1.7158185e-05 6.7072956e-06
  2.7444303e-06 2.2429303e-05 5.3592544e-06 4.3300664e-05 8.7391621e-05
  2.3328390e-05 9.8714358e-01 8.9324743e-04 3.5524289e-03 3.5106117e-04
  7.8302762e-03]]
Predicted class index: 11
The image is classified as a toxic plant: lilies
